In [1]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from lume_model.variables import ScalarVariable, DistributionVariable
from lume_model.models.gp_model import GPModel
from gpytorch.kernels import RBFKernel
from gpytorch.kernels import ScaleKernel

# Multi-output example

In [2]:
torch.manual_seed(0)

# Create training data, 1 input, 3 outputs
train_x = torch.rand(5, 1)
train_y = torch.stack(
    (
        torch.sin(train_x * (2 * torch.pi)) + 0.1 * torch.randn(1),
        torch.cos(train_x * (2 * torch.pi)) + 0.1 * torch.randn(1),
        torch.sin(2 * train_x * (2 * torch.pi)) + 0.1 * torch.randn(1),
    ),
    dim=-1,
).squeeze(1)


# Initialize the GP model
rbf_kernel = ScaleKernel(RBFKernel())

model = SingleTaskGP(
    train_x.to(dtype=torch.double),
    train_y.to(dtype=torch.double),
    covar_module=rbf_kernel,
)

# Fit the model
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll)

# Derive posterior mean and variance
model.eval()
test_x = torch.linspace(0, 1, 10).reshape(-1, 1).to(dtype=torch.double)
posterior = model.posterior(test_x)

# Derive the posterior mean and variance for each output
mean = posterior.mean
variance = posterior.variance
print("Posterior Mean for each output:")
print(mean)
print("\nPosterior Variance for each output:")
print(variance)

Posterior Mean for each output:
tensor([[ 0.2050,  0.8345,  0.4122],
        [ 0.6937,  0.7963,  1.0434],
        [ 1.0670,  0.1745,  0.3672],
        [ 0.8972, -0.4612, -0.7233],
        [ 0.3288, -0.9288, -0.3088],
        [-0.1707, -0.7832,  0.1967],
        [-0.6685, -0.1757,  0.0238],
        [-0.9328,  0.1558, -0.1630],
        [-0.5566,  0.2297,  0.0035],
        [-0.1373,  0.2452,  0.2042]], dtype=torch.float64,
       grad_fn=<CloneBackward0>)

Posterior Variance for each output:
tensor([[1.1023e-01, 1.3304e-01, 9.4012e-02],
        [7.6222e-04, 3.0346e-03, 4.2899e-04],
        [2.8662e-02, 3.8413e-02, 2.4013e-02],
        [1.1967e-02, 1.6607e-02, 1.0005e-02],
        [5.1544e-02, 5.3809e-02, 4.5043e-02],
        [1.0373e-01, 1.0429e-01, 9.1084e-02],
        [2.4203e-01, 2.3610e-01, 2.1332e-01],
        [4.6992e-03, 8.8862e-03, 3.6847e-03],
        [3.7522e-01, 3.6355e-01, 3.3097e-01],
        [5.7309e-01, 5.5293e-01, 5.0575e-01]], dtype=torch.float64,
       grad_fn=<CloneBac

## LUME-Model import

In [3]:
# Define input variables
input_variables = [ScalarVariable(name="x")]

# Define output variables
# Currently the "distribution_type" field doesn't do anything
output_variables = [
    DistributionVariable(name="output1", distribution_type="MultiVariateNormal"),
    DistributionVariable(name="output2", distribution_type="MultiVariateNormal"),
    DistributionVariable(name="output3", distribution_type="MultiVariateNormal"),
]

# Create lume_model instance
gp_lume_model = GPModel(
    model=model, input_variables=input_variables, output_variables=output_variables
)

### Evaluate model and run methods

In [4]:
input_dict = {"x": test_x.squeeze(1).to(dtype=torch.double)}

In [5]:
input_dict

{'x': tensor([0.0000, 0.1111, 0.2222, 0.3333, 0.4444, 0.5556, 0.6667, 0.7778, 0.8889,
         1.0000], dtype=torch.float64)}

In [6]:
# Evaluate function returns a dictionary mapping each output to a torch.distributions.Distribution
output_dict = gp_lume_model.evaluate(input_dict)

In [7]:
output_dict

{'output1': MultivariateNormal(loc: torch.Size([10]), covariance_matrix: torch.Size([10, 10])),
 'output2': MultivariateNormal(loc: torch.Size([10]), covariance_matrix: torch.Size([10, 10])),
 'output3': MultivariateNormal(loc: torch.Size([10]), covariance_matrix: torch.Size([10, 10]))}

In [8]:
test_prob = output_dict["output1"].sample(torch.Size([2]))

In [9]:
print("Posterior mean:", output_dict["output1"].mean)
print("Posterior Variance ", output_dict["output1"].variance)
print("Log Likelihood", output_dict["output1"].log_prob(test_prob))
print("Rsample ", output_dict["output1"].rsample(torch.Size([3])))

Posterior mean: tensor([ 0.2050,  0.6937,  1.0670,  0.8972,  0.3288, -0.1707, -0.6685, -0.9328,
        -0.5566, -0.1373], dtype=torch.float64, grad_fn=<ExpandBackward0>)
Posterior Variance  tensor([0.1102, 0.0008, 0.0287, 0.0120, 0.0515, 0.1037, 0.2420, 0.0047, 0.3752,
        0.5731], dtype=torch.float64, grad_fn=<ExpandBackward0>)
Log Likelihood tensor([4.5356, 4.6654], dtype=torch.float64, grad_fn=<SubBackward0>)
Rsample  tensor([[ 1.0552e-01,  7.1237e-01,  1.2059e+00,  8.3028e-01,  1.0869e-01,
          7.1085e-03, -9.0223e-01, -9.2138e-01, -7.6623e-01, -6.2605e-01],
        [ 3.7840e-04,  6.9813e-01,  9.1871e-01,  1.0180e+00,  5.5303e-01,
         -7.8040e-01, -1.2117e+00, -9.9632e-01, -1.2616e+00, -7.7939e-01],
        [-4.3100e-01,  6.9217e-01,  9.9770e-01,  8.6105e-01,  3.0568e-01,
         -2.0983e-01, -9.1296e-01, -8.8088e-01,  6.1089e-02,  2.8746e-01]],
       dtype=torch.float64, grad_fn=<AddBackward0>)


### Outputs with original model

In [10]:
print("Posterior mean:", posterior.mean[:, 0])
print("Posterior Variance ", posterior.variance[:, 0])

Posterior mean: tensor([ 0.2050,  0.6937,  1.0670,  0.8972,  0.3288, -0.1707, -0.6685, -0.9328,
        -0.5566, -0.1373], dtype=torch.float64, grad_fn=<SelectBackward0>)
Posterior Variance  tensor([0.1102, 0.0008, 0.0287, 0.0120, 0.0515, 0.1037, 0.2420, 0.0047, 0.3752,
        0.5731], dtype=torch.float64, grad_fn=<SelectBackward0>)


# A 3D Rosenbrock example for GPModel class

## Create and train a GP

In [11]:
# Define the 3D Rosenbrock function
def rosenbrock(X):
    x1, x2, x3 = X[..., 0], X[..., 1], X[..., 2]
    return (
        (1 - x1) ** 2
        + 100 * (x2 - x1**2) ** 2
        + (1 - x2) ** 2
        + 100 * (x3 - x2**2) ** 2
    )

In [13]:
# Generate training data
train_x = torch.rand(20, 3) * 4 - 2  # 20 points in 3D space, scaled to [-2, 2]
train_y = rosenbrock(train_x).unsqueeze(-1)  # Compute the Rosenbrock function values

# Define the GP model
gp_model = SingleTaskGP(train_x.to(dtype=torch.double), train_y.to(dtype=torch.double))

# Fit the model
mll = ExactMarginalLogLikelihood(gp_model.likelihood, gp_model)
fit_gpytorch_mll(mll)

# Evaluate the model on test data
test_x = torch.rand(10, 3) * 4 - 2  # 10 new points in 3D space
gp_model.eval()
posterior = gp_model.posterior(test_x)

# Get the mean and variance of the posterior
mean = posterior.mean
variance = posterior.variance

print("Posterior mean: ", mean)
print("Posterior variance: ", variance)

Posterior mean:  tensor([[ 589.6488],
        [ 414.3726],
        [ 379.7021],
        [ -63.6949],
        [1231.5723],
        [ -96.9024],
        [1341.5255],
        [-258.7856],
        [ 980.1755],
        [ 612.8108]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Posterior variance:  tensor([[72240.6283],
        [27952.7022],
        [ 3173.3891],
        [17167.4324],
        [11905.8578],
        [10077.6879],
        [ 4033.4934],
        [ 6140.0747],
        [17327.2274],
        [58078.6172]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)


## LUME-Model import

In [14]:
# Define input variables
input_variables = [
    ScalarVariable(name="x1"),
    ScalarVariable(name="x2"),
    ScalarVariable(name="x3"),
]

# Define output variables
# Currently the "distribution_type" field doesn't do anything
output_variables = [
    DistributionVariable(name="output1", distribution_type="MultiVariateNormal")
]

# Create lume_model instance
gp_lume_model = GPModel(
    model=gp_model, input_variables=input_variables, output_variables=output_variables
)

#### Evaluate model and run methods

In [15]:
input_x = torch.rand(3, 3) * 4 - 2  # 3 new points in 3D space
input_dict = {
    "x1": input_x[:, 0].to(dtype=torch.double),
    "x2": input_x[:, 1].to(dtype=torch.double),
    "x3": input_x[:, 2].to(dtype=torch.double),
}

In [16]:
# Evaluate function returns a dictionary mapping each output to a torch.distributions.Distribution
lume_dist = gp_lume_model.evaluate(input_dict)["output1"]

In [17]:
rand_test = torch.rand(1, 3)

In [18]:
print("Posterior mean:", lume_dist.mean)
print("Posterior Variance ", lume_dist.variance)
print("Log Likelihood", lume_dist.log_prob(rand_test))
print("Rsample ", lume_dist.rsample(torch.Size([3])))

Posterior mean: tensor([ 612.0247, 1552.4470, 1511.9490], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)
Posterior Variance  tensor([ 3687.1288, 92138.0799, 12148.2341], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)
Log Likelihood tensor([-161.8829], dtype=torch.float64, grad_fn=<SubBackward0>)
Rsample  tensor([[ 627.1446, 1884.3308, 1591.7666],
        [ 616.3797, 1960.4848, 1527.1977],
        [ 643.1253, 1825.0287, 1611.2168]], dtype=torch.float64,
       grad_fn=<AddBackward0>)


### Outputs with original model

In [19]:
posterior = gp_model.posterior(input_x)
botorch_dist = posterior.distribution

In [20]:
print("Posterior mean:", botorch_dist.mean)
print("Posterior Variance ", botorch_dist.variance)
print("Log Likelihood", botorch_dist.log_prob(rand_test))
print("Rsample ", botorch_dist.rsample(torch.Size([3])))

Posterior mean: tensor([ 612.0247, 1552.4470, 1511.9490], dtype=torch.float64,
       grad_fn=<AddBackward0>)
Posterior Variance  tensor([ 3687.1288, 92138.0799, 12148.2341], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)
Log Likelihood tensor([-161.8829], dtype=torch.float64, grad_fn=<SubBackward0>)
Rsample  tensor([[ 567.7197, 1495.8050, 1409.5655],
        [ 470.0016, 1833.1275, 1506.4477],
        [ 628.6214, 1976.9459, 1510.3016]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)
